In [1]:
import pandas as pd
import numpy as np
import os
from os import listdir
from os.path import isfile, join
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [59]:
PSA_Pokemon_df = pd.read_csv('PWCC_Pokemon.csv')
PSA_Pokemon_df_9 = PSA_Pokemon_df[(PSA_Pokemon_df.Rating == 9)]
PSA_Pokemon_df_10 = PSA_Pokemon_df[(PSA_Pokemon_df.Rating == 10)]

In [61]:
# File names from csv
uuid_9 = [[y.split('[')[1].split(']')[0].split(', ')[x][1:-1] for x in range(len(y.split('[')[1].split(']')[0].split(', ')))] for y in list(PSA_Pokemon_df_9.uuid)]
uuid_10 = [[y.split('[')[1].split(']')[0].split(', ')[x][1:-1] for x in range(len(y.split('[')[1].split(']')[0].split(', ')))] for y in list(PSA_Pokemon_df_10.uuid)]


# File names from directory
available_nines = [f.split('.jpg')[0] for f in listdir('PWCC/9') if isfile(join('PWCC/9', f))]
available_tens = [f.split('.jpg')[0] for f in listdir('PWCC/10') if isfile(join('PWCC/10', f))]


# Only use samples where both photos(front and back) exist in directory
nines_front = []
nines_back = []
for i in uuid_9:
    if (i[0] in available_nines) and (i[0] in available_nines):
        nines_front.append('PWCC/9/'+str(i[0])+'.jpg')
        nines_back.append('PWCC/9/'+str(i[1])+'.jpg')
        
tens_front = []
tens_back = []
for i in uuid_10:
    if (i[0] in available_tens) and (i[0] in available_tens):
        tens_front.append('PWCC/10/'+str(i[0])+'.jpg')
        tens_back.append('PWCC/10/'+str(i[1])+'.jpg')

In [62]:
sample_size = min(len(nines_front), len(tens_front))

image_front = nines_front[:sample_size] + tens_front[:sample_size]
image_back = nines_back[:sample_size] + tens_back[:sample_size]

y = [1 for i in tens_front[:sample_size]] + [0 for i in nines_front[:sample_size]]
y = pd.DataFrame(y)
y.value_counts()

0    13654
1    13654
dtype: int64

In [63]:
from tqdm.notebook import tqdm
from PIL import Image

res = 224

# Load front and back from paths and concat horizontally
def load_image(fr,ba):
    image_fr = Image.open(fr).convert('LA').resize((res, res))
    image_ba = Image.open(ba).convert('LA').resize((res, res))
    img = np.concatenate((image_fr, image_ba), axis = 1)
    arr = np.array(img)
    return arr

images = np.array([load_image(image_front[i],image_back[i]) for i in tqdm(range(len(image_front)))])

  0%|          | 0/27308 [00:00<?, ?it/s]

In [64]:
from sklearn.model_selection import train_test_split

x_tr, x_vl, y_tr, y_vl = train_test_split(
    images,
    y,
    test_size=0.2)

In [72]:
from __future__ import absolute_import, division, print_function, unicode_literals
import collections
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, GRU, LSTM, Bidirectional, TimeDistributed, Attention, Dropout
from tensorflow.keras.optimizers import Adam, SGD

in_front = keras.Input(batch_shape=(None, (res), (res*2), 2))

#Front
cov_0 = Conv2D(32, (5, 5), activation='relu')(in_front)
pool_0 = MaxPool2D((2, 2))(cov_0)
cov_1 = Conv2D(64, (5, 5), activation='relu')(pool_0)
pool_1 = MaxPool2D((2, 2))(cov_1)
cov_2 = Conv2D(64, (5, 5), activation='relu')(pool_1)
pool_2 = MaxPool2D((2, 2))(cov_2)
cov_3 = Conv2D(64, (5, 5), activation='relu')(pool_2)
pool_3 = MaxPool2D((2, 2))(cov_3)

# I usually don't use this layer
cov_4 = Conv2D(128, (5, 5), activation='relu')(pool_3)
pool_4 = MaxPool2D((6, 6))(cov_4)


flatten = Flatten()(pool_4)


fc = Dense(64, activation='relu')(flatten)
pred = Dense(1, activation='sigmoid')(fc)

model = keras.Model(
    inputs={
        'in_front': in_front
    },
    outputs=pred,
)

In [73]:
model.compile(
    optimizer=Adam(lr=0.0003),
    loss='binary_crossentropy',
    metrics=['accuracy', 'AUC']
)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 448, 2)]     0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 220, 444, 32)      1632      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 110, 222, 32)      0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 106, 218, 64)      51264     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 53, 109, 64)       0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 49, 105, 64)       102464    
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 24, 52, 64)        0   

In [74]:
history = model.fit(
    x={
        'in_front': x_tr
    },
    y=y_tr,
    
    epochs=100,
    batch_size=32,

    validation_data=(
        {
            'in_front': x_vl
        },
        y_vl
    ),
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    ],
    
    verbose=1

)

Epoch 1/100
683/683 [==============================] - 1767s 3s/step - loss: 1.0000 - accuracy: 0.5738 - auc: 0.5978 - val_loss: 0.6437 - val_accuracy: 0.6130 - val_auc: 0.7112
Epoch 2/100
683/683 [==============================] - 1715s 3s/step - loss: 0.6206 - accuracy: 0.6448 - auc: 0.7061 - val_loss: 0.6258 - val_accuracy: 0.6243 - val_auc: 0.7367
Epoch 3/100
683/683 [==============================] - 1718s 3s/step - loss: 0.6008 - accuracy: 0.6635 - auc: 0.7317 - val_loss: 0.5954 - val_accuracy: 0.6692 - val_auc: 0.7349
Epoch 4/100
683/683 [==============================] - 1726s 3s/step - loss: 0.5914 - accuracy: 0.6708 - auc: 0.7435 - val_loss: 0.6139 - val_accuracy: 0.6679 - val_auc: 0.7481
Epoch 5/100
683/683 [==============================] - 1702s 2s/step - loss: 0.5780 - accuracy: 0.6873 - auc: 0.7606 - val_loss: 0.5866 - val_accuracy: 0.6748 - val_auc: 0.7596
Epoch 6/100
683/683 [==============================] - 1704s 2s/step - loss: 0.5705 - accuracy: 0.6934 - auc: 0.766

In [76]:
def sig(x):
    if x>0.5:
        return 1
    else:
        return 0

vl_prediction = model.predict({'in_front': x_vl})
vl_prediction = [sig(x) for x in [x[0] for x in list(vl_prediction)]]
conf_matrix = tf.math.confusion_matrix(list(y_vl[0]),vl_prediction)
conf_matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[2234,  540],
       [1114, 1574]])>